In [14]:
import numpy as np

from qiskit import Aer, IBMQ
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import *
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.providers.aer.noise import NoiseModel

provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-07-07 12:21:43,801: Credentials are already in use. The existing account in the session will be replaced.


In [15]:
def get_operator(J,h,n):    
    pauli_list = []

    for (i,j) in J: # For each coefficient in J (couplings) we add a term J[i,j]Z_iZj
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True 
        z_p[n-1-j] = True
        pauli_list.append([J[(i,j)],Pauli(z_p, x_p)])
     
    for i in h: # For each coefficient in h we add a term h[i]Z_i
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True
        pauli_list.append([h[i],Pauli(z_p, x_p)])
    
    return WeightedPauliOperator(paulis=pauli_list)

In [9]:
# Edges of the graph

J1 = {(0,1):1, (1,2):1, (2,3):1, (3,4):1, (4,0):1}
h1 = {}
n = 5

# Hamiltonian

q_op =get_operator(J1,h1,n) 
print(q_op)
q_op.print_details()

Representation: paulis, qubits: 5, size: 5


/tmp/ipykernel_73/3746474484.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_p = np.zeros(n, dtype=np.bool)
/tmp/ipykernel_73/3746474484.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z_p = np.zeros(n, dtype=np.bool)
/opt/conda/lib/python3.8/site-packages/qiskit/quantum_info/operators/symplectic/pauli.py:185: DeprecationWarning: Initiali

'ZZIII\t(1+0j)\nIZZII\t(1+0j)\nIIZZI\t(1+0j)\nIIIZZ\t(1+0j)\nZIIIZ\t(1+0j)\n'

In [16]:
rep = 10
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

/opt/conda/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',


In [10]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Traceback (most recent call last):
  File "/tmp/ipykernel_73/406211574.py", line 7, in <module>
    result = qaoa.run(quantum_instance)
NameError: name 'quantum_instance' is not defined

Use %tb to get the full traceback.


/opt/conda/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refactored to qiskit.algorithms.VariationalAlgorithm (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_class('aqua.algorithms.VQAlgorithm',
/opt/conda/lib/python3.8/site-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For mo

In [11]:

p = 2
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Traceback (most recent call last):
  File "/tmp/ipykernel_73/2783247012.py", line 7, in <module>
    result = qaoa.run(quantum_instance)
NameError: name 'quantum_instance' is not defined

Use %tb to get the full traceback.


In [12]:
rep = 10
backendIBM = provider.get_backend('ibmq_ourense')
noise_model = NoiseModel.from_backend(backendIBM)
coupling_map = backendIBM.configuration().coupling_map
basis_gates = noise_model.basis_gates
backend = Aer.get_backend("qasm_simulator")


shots = 8192
optimization_level = 3
p = 1
quantum_instance = QuantumInstance(backend, shots = shots, 
                                    optimization_level = optimization_level,
                                    noise_model = noise_model,
                                    basis_gates = basis_gates,
                                    coupling_map = coupling_map)

Traceback (most recent call last):
  File "/tmp/ipykernel_73/3852027961.py", line 2, in <module>
    backendIBM = provider.get_backend('ibmq_ourense')
  File "/opt/conda/lib/python3.8/site-packages/qiskit/providers/provider.py", line 53, in get_backend
    raise QiskitBackendNotFoundError('No backend matches the criteria')
QiskitBackendNotFoundError: 'No backend matches the criteria'

Use %tb to get the full traceback.


In [13]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Traceback (most recent call last):
  File "/tmp/ipykernel_73/406211574.py", line 7, in <module>
    result = qaoa.run(quantum_instance)
NameError: name 'quantum_instance' is not defined

Use %tb to get the full traceback.
